# Structure Seach
#### Using the tracrRNAs found in Step 2, look for the consensus structure of each tracrRNA in the population.

# Keep this for the generete consensus method, If i want to take Step 3 results and do a deeper search

In [1]:
%load_ext autoreload
%autoreload 2

#Imports
import sys
sys.path.append("scripts/")
from AnnotateTree import AnnotateTree
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.SeqIO import parse, write
from collections import Counter
from easyFunctions import *
from InfernalResults import TRACR_RNA, ProcessInfernal
from StructureSearch import Main as StructureSearch
from os import chdir,system
from pickle import load,dump
from random import randint

#Global processes
def generate_consensus_struct_seqs_from_struct_results(infSearch,round_num):
    print "Generating Consensus Seqs For Round %i:" % (round_num)
    system("mkdir data/conseqs%i" % (round_num))
    structs = {}
    count = 0
    for ID,info in infSearch.iteritems():
        for hitset in info:
            for hit in hitset:
                if hit.location.start < hit.location.end:
                    try: structs[hit.structName].append(records[ID][hit.location.start-1:hit.location.end])
                    except: structs[hit.structName]= [records[ID][hit.location.start-1:hit.location.end]]
                else:
                    try: structs[hit.structName].append(records[ID][hit.location.end-1:hit.location.start])
                    except: structs[hit.structName]= [records[ID][hit.location.end-1:hit.location.start]]
    print "TracrName","#Seqs", "#UniqSeqs"
    for sName, seqs in structs.iteritems():
        newFasta = MakeFasta("data/conseqs%i/%s.fa" % (round_num, sName))
        counter = 0
        uniqSeqs = set(seqs)
        for seq in uniqSeqs:
            newFasta.write(sName.replace("_consensus","") + "_%i" % counter,seq)
            counter += 1
        print "\t",sName,len(seqs),len(uniqSeqs)
        newFasta.close()
    return "data/conseqs%i" % (round_num)

def FilterConSeqs(seqSet,referenceSeq,name):
    retSet = set()
    for seq in seqSet:
        percIdent1 = CalcPercIdent(referenceSeq,seq)
        percIdent2 = CalcPercIdent(referenceSeq,str(Seq(seq).reverse_complement()))
        if percIdent1 > 65.0 or percIdent2 > 65.0:
            if percIdent2 > percIdent1:seq = str(Seq(seq).reverse_complement())
            retSet.add(seq)       
    return retSet
r = lambda: randint(0,255)
def color():return '#%02X%02X%02X' % (r(),r(),r())

#Global variables 
records={}
for rec in parse("data/CasRelatedAssemblies.fasta", "fasta"): records[rec.id] = str(rec.seq)           
print "Number of Cas9s:",len(records)    

Number of Cas9s: 2023


In [2]:
print "Running Structural search"
#StructureSearch(0) 
print "Parsing structural result output"
chdir("/mnt/research/germs/shane/transActRNA/data")
infernalResults = ProcessInfernal(2)
chdir("/mnt/research/germs/shane/transActRNA/data")

# for roundNum in range(2,5):
#     print "Starting Round:",roundNum + 1
#     #Load the previous round results
#     fileName = "data/pickles/InfernalResults0%i.p" % (roundNum)
#     infernalSearch = load(open(fileName,"rb"))
#     dupHitsInAssembly = 0
#     for ID,structResults in infernalSearch.iteritems():
#         if len(structResults)>1: dupHitsInAssembly+=1
#     print "Round %i Results" % (roundNum)
# #     print "\t",file
#     print "\tNumber of assemblies covered:", len(infernalSearch.keys())
#     print "\tNumber of assemblies w/more than one struct hit:",dupHitsInAssembly
#     break
#     #Generate new consensus seqs
#     newConsDir = generate_consensus_struct_seqs_from_struct_results(infernalSearch,roundNum+1)
#     chdir(newConsDir)
#     #Process the next round of infernal results from the conseqs
#     print "Running Structural search"
#     StructureSearch(roundNum+1)
#     print "Parsing structural result output"
#     ProcessInfernal(roundNum+1)
#     chdir("")

Running Structural search
Parsing structural result output
Dumping /mnt/research/germs/shane/transActRNA/data/pickles/InfernalResults02.p Number of results: 833


In [8]:
chdir("../../")

In [3]:
%%bash
pwd

/mnt/research/germs/shane/transActRNA


In [3]:
colorCodes={}
TreeColors={}
for res in infernalResults:
    hitList = list(infernalResults[res])
    hitList.sort()
    try:TreeColors[res] = colorCodes[hitList[0]]
    except:
        colorCodes[hitList[0]] = color()
        TreeColors[res] = colorCodes[hitList[0]]
import pickle
pickle.dump(TreeColors, open("pickles/CondensedTreeColors.p","wb"))

In [ ]:
CasTree = AnnotateTree([],tree="data/Cas9-Like-clustered.faa.tree",colors=TreeColors)
CasTree.tree.render(file_name="InitResults.jpeg", w=1000, tree_style=CasTree.ts)
print "dsfsf"